# Mean and standart deviation for large array

In [1]:
import pyopencl
import numpy
from pyopencl import array as cla
from pyopencl.reduction import ReductionKernel

In [2]:
ctx = pyopencl.create_some_context()
queue = pyopencl.CommandQueue(ctx, properties=pyopencl.command_queue_properties.PROFILING_ENABLE)
print(ctx)

<pyopencl.Context at 0x564244d0c530 on <pyopencl.Device 'TITAN V' on 'NVIDIA CUDA' at 0x5642444c5ee0>>


In [3]:
shape = (2048, 2048)
size = numpy.prod(shape)
data = 2**(numpy.random.random(shape).astype("float32")*16)
data_d = cla.to_device(queue, data)
res_d = cla.zeros(queue, shape=8, dtype="float32")
print(numpy.mean(data), numpy.std(data))

5907.7095 12596.121


In [4]:
rk1 = ReductionKernel(ctx, dtype_out="float32", neutral=0, reduce_expr="a+b", map_expr="x[i]", 
                      arguments="__global float *x", name="simple_sum_kernel", )

In [5]:
_, evt = rk1(data_d, return_event=True, out=res_d[0])

In [6]:
1e-6*(evt.profile.end-evt.profile.start)

0.015359999999999999

In [7]:
res_d[0]/size

array(5907.70996094)

In [8]:
arguments = ("__global float *value",)
float2 = pyopencl.tools.get_or_register_dtype("float2")
map_expr = "isfinite(value[i]) ? (float2)(value[i], 1.0f) : (float2)(0.0f, 0.0f)"
reduce_expr = "a+b"
neutral = "(float2)(0.0f,0.0f)"
#output_statement = "value[i] = item.s0; index[i+1] = item.s1;"
rk2 = ReductionKernel(ctx, dtype_out=float2, neutral=neutral, reduce_expr=reduce_expr, 
                      map_expr=map_expr, 
                      arguments=arguments, 
                      name="counter_sum_kernel", )

In [9]:
res, evt = rk2(data_d, return_event=True)

In [10]:
m = res.get()
print(m)
print(size)
print("mean", numpy.mean(data), m["s0"]/m["s1"])

(2.4778732e+10, 4194304.)
4194304
mean 5907.7095 5907.71


In [11]:
size

4194304

In [12]:
%load_ext pyopencl.ipython_ext


In [35]:
%%cl_kernel

kernel void mean2std(global float* value,
                     global float2* mean,
                     global float* delta2,
                     int size)
{
    int gid = get_global_id(0);
    if (gid>=size)
    {
       return;
    }
    float m = mean[0].s0/mean[0].s1;
    float delta = value[gid] - m;
    delta2[gid] = delta*delta;
}

kernel void sigmaclip(global float* value,
                       global float2* mean,
                       global float2* std,
                       float cutof,                 
                       int size)
{
    int gid = get_global_id(0);
    if (gid>=size)
    {
       return; 
    }
       
    float m = mean[0].s0/mean[0].s1;
    float s = sqrt(std[0].s0/(std[0].s1-1.0f));
    if (fabs(value[gid]-m)>(cutof*s))
    {
        value[gid] = NAN;
    }
}

In [36]:
delta2_d = cla.zeros_like(data_d)
evt = mean2std(queue, (size,), None, data_d.data, res.data, delta2_d.data, numpy.int32(size))
evt.wait()
print(1e-6*(evt.profile.end-evt.profile.start))

0.07168


In [37]:
delta2_d

array([[3.4863436e+07, 3.4627712e+07, 1.8556748e+07, ..., 2.2372484e+07,
        3.4601048e+07, 9.6400430e+06],
       [1.8533979e+08, 3.4759348e+07, 1.2853121e+09, ..., 3.4836780e+07,
        3.4322756e+07, 2.7519424e+07],
       [3.4800984e+07, 3.3533030e+09, 3.4374776e+07, ..., 2.9007504e+08,
        3.4854928e+07, 3.3851220e+07],
       ...,
       [3.4446456e+07, 2.1152948e+07, 3.0446658e+07, ..., 2.1082941e+09,
        1.9726779e+06, 3.4835512e+07],
       [3.4315688e+07, 3.4580804e+07, 2.9846770e+07, ..., 3.3242342e+07,
        3.9068262e+06, 6.3158227e+08],
       [3.4709964e+07, 3.4675940e+07, 2.7183690e+08, ..., 3.4869888e+07,
        3.4863356e+07, 3.4742696e+07]], dtype=float32)

In [38]:
res2, evt = rk2(delta2_d, return_event=True)
v=res2.get()
print(1e-6*(evt.profile.end-evt.profile.start))
print(res2)
std = numpy.sqrt(v["s0"]/(v["s1"]-1.0))
print("std", numpy.std(data), std)

0.012288
(6.654778e+14, 4194304.)
std 12596.121 12596.122809342662


In [39]:
evt = sigmaclip(queue, (size,), None, data_d.data, res.data, res2.data, numpy.float32(1.0), numpy.int32(size))
evt.wait()
print(1e-6*(evt.profile.end-evt.profile.start))

0.07168


In [40]:
def sigma_clip_np(data, cutof=3):
    ldata = data.copy()
    first_size = current_size = numpy.isfinite(ldata).sum()
    last_size = current_size+1

    while last_size>current_size:
        last_size = current_size
        m = numpy.nanmean(ldata)
        s = numpy.nanstd(ldata)
        ldata[abs(ldata-m)>cutof*s] = numpy.nan
        current_size = numpy.isfinite(ldata).sum()
        print(current_size)
    print(first_size, current_size)
    return m,s
        

In [41]:
%%time 
sigma_clip_np(data, 3)

4041051
3894503
3754402
3620746


/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater
  # Remove the CWD from sys.path while we load stuff.


3493631
3372566
3257552
3148622
3045353
2947655
2855471
2768651
2686643
2609196
2536440
2468420
2404553
2344450
2288517
2236248
2187417
2142031
2099872
2060240
2023477
1989588
1958134
1928924
1901844
1876727
1853340
1831613
1811543
1792877
1775451
1759254
1744347
1730638
1718002
1706116
1695227
1685207
1675996
1667461
1659572
1652296
1645592
1639314
1633560
1628231
1623180
1618541
1614190
1610176
1606521
1603196
1600161
1597306
1594681
1592213
1589955
1587896
1585990
1584198
1582580
1581108
1579687
1578397
1577139
1575925
1574763
1573677
1572714
1571848
1571013
1570272
1569595
1569015
1568480
1567987
1567536
1567123
1566719
1566360
1566045
1565753
1565534
1565317
1565130
1564939
1564754
1564594
1564452
1564323
1564184
1564065
1563942
1563821
1563715
1563633
1563561
1563485
1563425
1563384
1563333
1563285
1563242
1563199
1563155
1563122
1563078
1563031
1562989
1562942
1562890
1562832
1562787
1562745
1562701
1562666
1562639
1562616
1562590
1562569
1562548
1562521
1562491
1562461
1562437


(14.842549, 15.78905)

In [44]:
def sigma_clip_cl(data, cutof=3):
    data_d.set(data)
    res, evt = rk2(data_d, return_event=True)
    mm = res.get()
    first_size = current_size = mm["s1"]
    last_size = current_size +1
    while last_size>current_size:
        evt2 = mean2std(queue, (size,), None, data_d.data, res.data, delta2_d.data, numpy.int32(size))
        res2, evt3 = rk2(delta2_d, return_event=True, wait_for=[evt2])
        evt4 = sigmaclip(queue, (size,), None, data_d.data, res.data, res2.data, numpy.float32(cutof), numpy.int32(size))
        m = mm["s0"]/mm["s1"]
        v = res2.get()
        s = numpy.sqrt(v["s0"]/(v["s1"]-1.0))        
        last_size = current_size
        res, evt = rk2(data_d, return_event=True)
        mm = res.get()
        current_size = mm["s1"]
        print(current_size)

    print(first_size, current_size)
    return m,s

In [45]:
%%time 
sigma_clip_cl(data, 3)

4041051.0
3894503.0
3754402.0
3620746.0
3493631.0
3372565.0
3257551.0
3148619.0
3045351.0
2947653.0
2855464.0
2768643.0
2686636.0
2609189.0
2536437.0
2468419.0
2404552.0
2344450.0
2288517.0
2236248.0
2187417.0
2142031.0
2099872.0
2060240.0
2023477.0
1989588.0
1958134.0
1928924.0
1901844.0
1876728.0
1853340.0
1831613.0
1811543.0
1792877.0
1775451.0
1759254.0
1744347.0
1730638.0
1718002.0
1706116.0
1695227.0
1685207.0
1675996.0
1667461.0
1659572.0
1652296.0
1645592.0
1639314.0
1633560.0
1628231.0
1623180.0
1618541.0
1614190.0
1610176.0
1606521.0
1603196.0
1600161.0
1597306.0
1594681.0
1592213.0
1589955.0
1587896.0
1585990.0
1584198.0
1582580.0
1581108.0
1579687.0
1578397.0
1577139.0
1575925.0
1574763.0
1573677.0
1572714.0
1571848.0
1571014.0
1570273.0
1569596.0
1569016.0
1568481.0
1567989.0
1567541.0
1567126.0
1566723.0
1566365.0
1566047.0
1565755.0
1565536.0
1565318.0
1565130.0
1564939.0
1564754.0
1564594.0
1564452.0
1564323.0
1564184.0
1564065.0
1563942.0
1563821.0
1563715.0
1563633.0


(14.842546, 15.789050583389274)

0